In [1]:
import cv2 as cv
import os
from os.path import exists
import numpy as np
import mediapipe as mp

objc[15145]: Class CaptureDelegate is implemented in both /Users/M374155/miniforge3/lib/python3.10/site-packages/cv2/cv2.abi3.so (0x15d7765a0) and /Users/M374155/miniforge3/lib/python3.10/site-packages/mediapipe/.dylibs/libopencv_videoio.3.4.16.dylib (0x15e7f8860). One of the two will be used. Which one is undefined.
objc[15145]: Class CVWindow is implemented in both /Users/M374155/miniforge3/lib/python3.10/site-packages/cv2/cv2.abi3.so (0x15d7765f0) and /Users/M374155/miniforge3/lib/python3.10/site-packages/mediapipe/.dylibs/libopencv_highgui.3.4.16.dylib (0x12c80ca68). One of the two will be used. Which one is undefined.
objc[15145]: Class CVView is implemented in both /Users/M374155/miniforge3/lib/python3.10/site-packages/cv2/cv2.abi3.so (0x15d776618) and /Users/M374155/miniforge3/lib/python3.10/site-packages/mediapipe/.dylibs/libopencv_highgui.3.4.16.dylib (0x12c80ca90). One of the two will be used. Which one is undefined.
objc[15145]: Class CVSlider is implemented in both /Users/M

## Copy files from orginal dataset

In [8]:
import os
import shutil

# Set the path of the parent directory
parent_directory = "/Users/M374155/Desktop/Preprared New Dataset"

# Create a new directory to copy the files to
new_directory = "/Users/M374155/Desktop/Test/NonSleepy"

# Create the new directory if it doesn't already exist
if not os.path.exists(new_directory):
    os.makedirs(new_directory)

# Counter for the new directory names
counter = 1
for partent_dir_numer in range (1, 20):
    directory = os.path.join(parent_directory, str(partent_dir_numer))
    # Loop through each subdirectory of the parent directory
    for root, dirs, files in os.walk(directory):
        for file in files:
            # Check if the file is called "sleepyCombination.avi"
            if file == "nonsleepyCombination.avi":
                # Create the new subdirectory for this file
                new_subdirectory = os.path.join(new_directory, str(counter))
                os.makedirs(new_subdirectory, exist_ok=True)
                # Copy the file to the new subdirectory
                shutil.copy2(os.path.join(root, file), os.path.join(new_subdirectory, file))
                # Increment the counter
                counter += 1


## Create folders for extracted values

In [3]:
DATA_PATH = os.path.join('Extracted_Values_Bigger_Confidence/')

for state in ["Sleepy", "NonSleepy"]:
    for number in range(1,90):
            try:
                os.makedirs(os.path.join(DATA_PATH, state , str(number)))
            except:
                pass


### In order to solve openCV problem use version 4.5.5.62

## Extract face features from video

In [4]:
def extractKeypoints(result):
    if result.multi_face_landmarks:
        for face_detected in result.multi_face_landmarks:
            face = np.array([[res.x, res.y, res.z] for res in face_detected.landmark]).flatten()
    else:
        face = np.zeros(1434)

    return face

In [6]:
def drawLandmarks(mp_face_mesh, results, frame): 
    mp_drawing = mp.solutions.drawing_utils  
    mp_drawing_styles = mp.solutions.drawing_styles
    if results.multi_face_landmarks:
        for face_landmarks in results.multi_face_landmarks:
            mp_drawing.draw_landmarks(image=frame, landmark_list=face_landmarks, connections=mp_face_mesh.FACEMESH_TESSELATION, landmark_drawing_spec=None, connection_drawing_spec=mp_drawing_styles.get_default_face_mesh_tesselation_style())
            mp_drawing.draw_landmarks(image=frame, landmark_list=face_landmarks, connections=mp_face_mesh.FACEMESH_CONTOURS, landmark_drawing_spec=None, connection_drawing_spec=mp_drawing_styles.get_default_face_mesh_contours_style())
            mp_drawing.draw_landmarks(image=frame, landmark_list=face_landmarks, connections=mp_face_mesh.FACEMESH_IRISES, landmark_drawing_spec=None, connection_drawing_spec=mp_drawing_styles.get_default_face_mesh_iris_connections_style())

In [6]:

mp_face_mesh = mp.solutions.face_mesh
mp_drawing = mp.solutions.drawing_utils  
mp_drawing_styles = mp.solutions.drawing_styles

folder_numbers = range(1,3)
drowsiness_level = np.array(['NonSleepy', 'Sleepy'])

for state in ["Sleepy", "NonSleepy"]:
    for number in folder_numbers:
            videoName = "sleepyCombination.avi" if state == "Sleepy" else "nonsleepyCombination.avi"
            videoPath = os.path.join('/Users/M374155/Desktop/Test/', state, str(number), videoName)
            cap = cv.VideoCapture(videoPath)
            frame_count = int(cap.get(cv.CAP_PROP_FRAME_COUNT))
            print(frame_count)
            with mp_face_mesh.FaceMesh(
                refine_landmarks=True,
                min_detection_confidence=0.6,
                min_tracking_confidence = 0.8) as mesh:

                for frame_number in range(frame_count):
                    ret, frame = cap.read()
                
                    image = cv.cvtColor(frame, cv.COLOR_BGR2RGB)  
                    image.flags.writeable = False 
                    results = mesh.process(image)
                    image.flags.writeable = True
                    image = cv.cvtColor(image, cv.COLOR_RGB2BGR)  

                    drawLandmarks(mp_face_mesh=mp_face_mesh, results=results, frame=image)

                    landmarks = extractKeypoints(results)
                    npyPath = os.path.join("./Extracted_Values/Sleepy", str(number), str(frame_number))
                    # np.save(npyPath, landmarks)

                    cv.imshow('Video', image)
                    if cv.waitKey(1) == ord('q'):
                        break
            cap.release()
            cv.destroyAllWindows()

1521


INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


2803
3330


[NULL @ 0x2a9bbc110] time_scale/num_units_in_tick invalid or unsupported (0/0)
[NULL @ 0x2a9bbc110] Overread VUI by 8 bits
[h264 @ 0x299182370] corrupted macroblock 34 4 (total_coeff=-1)
[h264 @ 0x299182370] error while decoding MB 34 4
[h264 @ 0x299182370] Invalid level prefix
[h264 @ 0x299182370] error while decoding MB 23 25
[h264 @ 0x29918b780] illegal short term buffer state detected


1513
2732
1497
2880
2934
1260
2679
3086
3073
1445
3302
1428
2959
2868
1405
2888
1426
2812


...

[NULL @ 0x11f9d50c0] time_scale/num_units_in_tick invalid or unsupported (0/0)
[NULL @ 0x11f9d50c0] Overread VUI by 8 bits
[h264 @ 0x29ff0dc70] Invalid level prefix
[h264 @ 0x29ff0dc70] error while decoding MB 29 4
[h264 @ 0x29ff0dc70] corrupted macroblock 28 25 (total_coeff=-1)
[h264 @ 0x29ff0dc70] error while decoding MB 28 25
[h264 @ 0x29ff30110] illegal short term buffer state detected
[NULL @ 0x11f9d50c0] time_scale/num_units_in_tick invalid or unsupported (0/0)
[h264 @ 0x29fff33b0] corrupted macroblock 29 19 (total_coeff=-1)
[h264 @ 0x29fff33b0] error while decoding MB 29 19
[h264 @ 0x29fff33b0] corrupted macroblock 27 10 (total_coeff=-1)
[h264 @ 0x29fff33b0] error while decoding MB 27 10
[h264 @ 0x29ff0dc70] illegal short term buffer state detected


1538
2882
2918
1420
2861
1555
4460


[NULL @ 0x2a9be19a0] missing picture in access unit with size 16
[h264 @ 0x29fffb210] No start code is found.
[h264 @ 0x29fffb210] Error splitting the input into NAL units.
[NULL @ 0x2a9be19a0] missing picture in access unit with size 16
[h264 @ 0x29fffda20] No start code is found.
[h264 @ 0x29fffda20] Error splitting the input into NAL units.
[NULL @ 0x2a9be19a0] missing picture in access unit with size 16
[h264 @ 0x29faaab90] No start code is found.
[h264 @ 0x29faaab90] Error splitting the input into NAL units.
[NULL @ 0x2a9be19a0] missing picture in access unit with size 16
[h264 @ 0x29ffe1220] No start code is found.
[h264 @ 0x29ffe1220] Error splitting the input into NAL units.
[NULL @ 0x2a9be19a0] missing picture in access unit with size 16
[h264 @ 0x29ff0e7b0] No start code is found.
[h264 @ 0x29ff0e7b0] Error splitting the input into NAL units.
[NULL @ 0x2a9be19a0] missing picture in access unit with size 16
[h264 @ 0x29ff0abb0] No start code is found.
[h264 @ 0x29ff0abb0] Erro

3168
1558
2946


: 